<a href="https://colab.research.google.com/github/surtweig/kpi-assignments/blob/master/reliability/reliability_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Схема

In [42]:
connectivity = [
    # 0 1 2 3 4 5 6 7 8 9
    [ 0,1,0,0,0,0,0,0,0,0 ], # 0 start node
    [ 0,0,1,1,0,0,0,0,0,0 ], # 1 
    [ 0,0,0,0,1,1,0,0,0,0 ], # 2
    [ 0,0,0,0,1,0,1,0,1,0 ], # 3 
    [ 0,0,0,0,0,1,1,0,1,0 ], # 4 
    [ 0,0,0,0,0,0,1,1,0,0 ], # 5 
    [ 0,0,0,0,0,0,0,1,1,0 ], # 6 
    [ 0,0,0,0,0,0,0,0,0,1 ], # 7 
    [ 0,0,0,0,0,0,0,0,0,1 ], # 8
    [ 0,0,0,0,0,0,0,0,0,0 ]  # 9 end node
]

probabilities = [1, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.92, 0.94, 1]

time = 10

### Поиск путей прохода

In [43]:
def findRoutes(start, finish, con, routesList, currentRoute = None):
    if currentRoute == None:
        currentRoute = []

    if len(currentRoute) > 0 and currentRoute[-1] == finish:
        routesList.append(currentRoute)
        return
    
    if len(currentRoute) == 0:
        currentRoute.append(start)

    currentNode = currentRoute[-1]

    for nextNode in range(len(con[currentNode])):
        if con[currentNode][nextNode]:
            newRoute = list(currentRoute)
            newRoute.append(nextNode)
            findRoutes(start, finish, con, routesList, newRoute)

routesList = []
findRoutes(0, len(probabilities)-1, connectivity, routesList)

print(len(routesList))
for route in routesList:
    print(route)

18
[0, 1, 2, 4, 5, 6, 7, 9]
[0, 1, 2, 4, 5, 6, 8, 9]
[0, 1, 2, 4, 5, 7, 9]
[0, 1, 2, 4, 6, 7, 9]
[0, 1, 2, 4, 6, 8, 9]
[0, 1, 2, 4, 8, 9]
[0, 1, 2, 5, 6, 7, 9]
[0, 1, 2, 5, 6, 8, 9]
[0, 1, 2, 5, 7, 9]
[0, 1, 3, 4, 5, 6, 7, 9]
[0, 1, 3, 4, 5, 6, 8, 9]
[0, 1, 3, 4, 5, 7, 9]
[0, 1, 3, 4, 6, 7, 9]
[0, 1, 3, 4, 6, 8, 9]
[0, 1, 3, 4, 8, 9]
[0, 1, 3, 6, 7, 9]
[0, 1, 3, 6, 8, 9]
[0, 1, 3, 8, 9]


### Поиск работоспособных состояний

In [44]:
# возвращает все варианты перебора s (множество всех подмножеств)
def superset(s, sstart = [], istart = 0):
    supers = [sstart]

    if istart < len(s):
        for i in range(istart, len(s)):
            sstart2 = list(sstart)
            sstart2.append(s[i])
            supers.extend(superset(s, sstart2, i+1))
    return supers

# считает вероятность конкретного состояния
def probMult(state, probabilities):
    p = 1
    for i in range(len(state)):
        if state[i] == 0:
            p *= 1-probabilities[i]
        else:
            p *= probabilities[i]
    return p

# считает хэш состояния, потому что Питон не умеет хэшировать листы
def stateHash(state):
    hash = 0
    mask = 1
    for i in range(len(state)):
        hash += mask*state[i]
        mask <<= 1
    return hash

# находим все работоспособные состояния и считаем их вероятность
viableStates = {}
for r in routesList:
    state = [1 if i in r else 0 for i in range(len(probabilities))]
    volatileNodes = []
    for i in range(len(state)):
        if state[i] == 0:
            volatileNodes.append(i)

    volatileNodesSuperSet = superset(volatileNodes)
    for vn in volatileNodesSuperSet:
        addedState = list(state)
        for j in vn:
            addedState[j] = 1
        viableStates[stateHash(addedState)] = (probMult(addedState, probabilities), addedState)

totalProbability = 0
for k,v in viableStates.items():
    totalProbability += v[0]
    print(k, v)

#print("\nTotal probability = ", totalProbability)


759 (0.0030404160000000032, [1, 1, 1, 0, 1, 1, 1, 1, 0, 1])
767 (0.007094304000000008, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1])
1023 (0.11114409600000001, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
1015 (0.047633184, [1, 1, 1, 0, 1, 1, 1, 1, 1, 1])
887 (0.0041420159999999975, [1, 1, 1, 0, 1, 1, 1, 0, 1, 1])
895 (0.009664703999999996, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1])
695 (0.00033782400000000024, [1, 1, 1, 0, 1, 1, 0, 1, 0, 1])
703 (0.0007882560000000005, [1, 1, 1, 1, 1, 1, 0, 1, 0, 1])
959 (0.012349343999999996, [1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
951 (0.0052925759999999985, [1, 1, 1, 0, 1, 1, 0, 1, 1, 1])
727 (0.0005365440000000007, [1, 1, 1, 0, 1, 0, 1, 1, 0, 1])
735 (0.0012519360000000012, [1, 1, 1, 1, 1, 0, 1, 1, 0, 1])
991 (0.019613664, [1, 1, 1, 1, 1, 0, 1, 1, 1, 1])
983 (0.008405856000000003, [1, 1, 1, 0, 1, 0, 1, 1, 1, 1])
855 (0.0007309439999999998, [1, 1, 1, 0, 1, 0, 1, 0, 1, 1])
863 (0.0017055359999999993, [1, 1, 1, 1, 1, 0, 1, 0, 1, 1])
791 (8.121599999999996e-05, [1, 1, 1, 0, 1, 0, 0, 0, 1, 1])
799

### Рассчет параметров распределения

In [45]:
import numpy as np

expLambda = -np.log(totalProbability) / time
expMeanTime = 1/expLambda

print("P = {0}, lambda = {1}, mean T = {2}".format(totalProbability, expLambda, expMeanTime))

P = 0.4346051119999999, lambda = 0.08333174486887193, mean T = 12.000228743242648
